In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)

import datetime

import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import seaborn as sns
sns.set(style="whitegrid")

In [2]:

def new_string(number):
    
    new_string = df.date[number].replace("T000000",'')
    
    new_string =  new_string[:4] + "-" + new_string[4:]
    
    new_string =  new_string[:7] + "-" + new_string[7:]
    
    return new_string

def into_time(string):
    return datetime.datetime.strptime(string, "%Y-%m-%d")

In [3]:
df = pd.read_csv('kc_house_data_test_features.csv')
df.rename(columns = {'Unnamed: 0': 'index'}, inplace = True)
df.set_index('index', inplace = True)

In [4]:
df['sold_time'] = [into_time(i) for i in [new_string(i) for i in range(len(df))]]
df['years_ago_sold'] = abs(pd.DatetimeIndex(df['sold_time']).year - 2021)

df = df.drop(['sold_time','date', 'id','zipcode', 'lat', 'long', "waterfront", 'grade'], axis = 1)

In [5]:
yr_renovated_minus_yr_built = []

for i in range(len(df)): 

    if df.yr_renovated[i] == 0:
        yr_renovated_minus_yr_built.append(0)
    else:
        yr_renovated_minus_yr_built.append(df.yr_renovated[i] - df.yr_built[i])

df["renovated_after"] = yr_renovated_minus_yr_built

df['yr_built'] = abs(df.yr_built- 2021)

In [6]:
dummy = pd.get_dummies(df.condition)

df_condition = pd.concat([dummy,df], axis = 1)

dummy = pd.get_dummies(df.condition)

df_condition = pd.concat([dummy,df], axis = 1)

df_condition = df_condition[[1,2,3,4,5]]

df_condition.columns = ['Condition_1',"Condition_2","Condition_3","Condition_4","Condition_5"]

df = pd.concat([df_condition,df], axis = 1)
#df = pd.concat([df_grade,df], axis = 1)

#drop for multicollinearity
df = df.drop(["Condition_3", 'condition'], axis = 1)

In [7]:
df.columns

Index(['Condition_1', 'Condition_2', 'Condition_4', 'Condition_5', 'bedrooms',
       'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15', 'years_ago_sold', 'renovated_after'],
      dtype='object')

In [8]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(df)
poly2_columns = poly_2.get_feature_names(df.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)

df_poly2.columns


Index(['Condition_1', 'Condition_2', 'Condition_4', 'Condition_5', 'bedrooms',
       'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       ...
       'sqft_living15^2', 'sqft_living15 sqft_lot15',
       'sqft_living15 years_ago_sold', 'sqft_living15 renovated_after',
       'sqft_lot15^2', 'sqft_lot15 years_ago_sold',
       'sqft_lot15 renovated_after', 'years_ago_sold^2',
       'years_ago_sold renovated_after', 'renovated_after^2'],
      dtype='object', length=189)

In [9]:
df_poly2.shape


(4323, 189)

In [10]:
import pickle

# Load the columns I need
with open('features.pickle', 'rb') as file:
    features = pickle.load(file)

with open('regression_model.pickle', 'rb') as f:
    lm = pickle.load(f)
    
with open('scaler.pickle', "rb") as f:
    scaler2 = pickle.load(f)

In [11]:
len(features)

100

In [12]:
len(features) - len(df_poly2.columns)

-89

In [13]:
#df_poly2 = df_poly2.drop(set(list(df_poly2.columns)) - set(list(features)),axis = 1)

In [14]:
#len(features) - len(df_poly2.columns)

In [15]:
df_to_fit = scaler2.transform(df_poly2[features])

In [16]:
lm.predict(df_to_fit)

array([264256.57997729, 258792.35198789, 445879.30062825, ...,
       406084.40353472, 443167.87629712, 405923.53266537])

In [27]:
pd.DataFrame(lm.predict(df_to_fit)).to_csv('Silverman_Phase2')

,0
0,264256.579977
1,258792.351988
2,445879.300628
3,331191.272704
4,336804.031253
...,...
4318,535215.190990
4319,313880.786404
4320,406084.403535
4321,443167.876297


In [26]:
savetxt('Silverman_Phase_2_predictions.csv', Silverman_Phase_2_predictions_array, delimiter=',')

',0\n0,264256.57997728605\n1,258792.3519878866\n2,445879.30062824924\n3,331191.2727038034\n4,336804.03125334624\n5,248822.3526437705\n6,528752.9972075642\n7,404581.446206269\n8,313600.8306404002\n9,392049.4823808289\n10,524125.4407929681\n11,486899.8685501782\n12,436635.1772616609\n13,300915.20886829577\n14,448770.7638068899\n15,313604.65035159123\n16,125676.32738375803\n17,300490.00385713426\n18,264525.7596656163\n19,329972.93564174057\n20,293435.4873762024\n21,609681.0431588561\n22,558306.8023436981\n23,445137.55661556125\n24,391480.9503009241\n25,119772.51047010109\n26,499401.65306137403\n27,316129.9461186669\n28,503477.0431993335\n29,378518.76459423313\n30,444375.21971517586\n31,323196.29942299915\n32,466669.46253167035\n33,310695.42715660465\n34,234312.88008428807\n35,389893.1721985121\n36,528254.5538890224\n37,362273.70871643454\n38,360896.0310849595\n39,317078.8291495863\n40,246461.6503613794\n41,314988.38185151084\n42,339696.37300253\n43,421923.3894659104\n44,320676.24469954753

In [17]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
Silverman_Phase_2_predictions_array = lm.predict(df_to_fit)
# save to csv file
savetxt('Silverman_Phase_2_predictions.csv', Silverman_Phase_2_predictions_array, delimiter=',')